In [70]:
import urllib.request as req
import json
import pandas as pd
import numpy as np
import requests

In [71]:
website_final = pd.DataFrame(np.array([['01', 'hahow', 'https://hahow.in/courses']
                                 , ['02', 'tibame', 'https://www.tibame.com/courselibrary']
                                 , ['03', 'pressplay', 'https://www.pressplay.cc/']
                                 , ['04', 'yotta', 'https://www.yottau.com.tw/home']])
                                 , columns = ['w_id', 'w_name', 'w_url'])
categories_final = pd.DataFrame(np.array([['01', '語言']
                                    , ['02', '藝術']
                                    , ['03', '設計']
                                    , ['04', '多媒體設計']
                                    , ['05', '程式']
                                    , ['06', '行銷']
                                    , ['07', '投資理財']
                                    , ['08', '職場技能']
                                    , ['09', '生活品味']])
                                    , columns = ['cg_id', 'cg_name'])
status_final = pd.DataFrame(np.array([['01', 'SUCCESS']
                                , ['02', 'PUBLISHED']
                                , ['03', 'INCUBATING']])
                                , columns = ['status_id', 'status_description'])

In [72]:
courses_final = pd.DataFrame(columns = ['c_id', 'c_name', 'w_id', 'c_description', 'c_price', 'c_total_time', 't_id', 'c_start_time', 'c_buyers', 'cg_id', 'c_url', 'c_img_url', 'status_id'])
teachers_final = pd.DataFrame(columns = ['t_id', 't_name', 't_introduction'])
reviews_final = pd.DataFrame(columns = ['c_id', 'r_id', 'r_statrs', 'r_content'])
discounts_final = pd.DataFrame(columns = ['c_id', 'd_id', 'd_content'])
funding_final = pd.DataFrame(columns = ['f_id', 'f_due_date', 'f_process', 'c_id'])

In [73]:
course_count = 0
funding_count = 0
review_count = 0
discount_count = 0

# Hahow

## Tables

In [74]:
courses = pd.DataFrame(columns = ['c_id', 'c_name', 'w_id', 'c_description', 'c_price', 'c_total_time', 't_id', 'c_start_time', 'c_buyers', 'cg_id', 'c_url', 'c_img_url', 'status_id'])
teachers = pd.DataFrame(columns = ['t_id', 't_name', 't_introduction'])
reviews = pd.DataFrame(columns = ['c_id', 'r_id', 'r_statrs', 'r_content'])
discounts = pd.DataFrame(columns = ['c_id', 'd_id', 'd_content'])
funding = pd.DataFrame(columns = ['f_id', 'f_due_date', 'f_process', 'c_id'])

## Web_crawler

In [75]:
for i in range(29):
    url = "https://api.hahow.in/api/courses?limit=24&page=" + str(i)
    request = req.Request(url, headers = {
        "Content-Type":"application/json; charset=utf-8",
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    })
    with req.urlopen(request) as response:
        result = response.read().decode("utf-8")
    result = json.loads(result)
    
    for j in range(len(result['data'])):
        #courses
        data_courses = {'c_id':str(course_count + 1).zfill(5)
                        , 'c_name':result['data'][j]['title']
                        , 'w_id':'01'
                        , 'c_description':result['data'][j]['metaDescription']
                        , 'c_price':result['data'][j]['price']
                        , 't_id':str(course_count + 1).zfill(5)
                        , 'c_buyers':result['data'][j]['numSoldTickets']
                        , 'c_url':"https://api.hahow.in/api/courses/" + result['data'][j]['_id']
                        , 'c_img_url':result['data'][j]['coverImage']['url']}
        courses = courses.append(data_courses, ignore_index = True)
        #teahcers
        if result['data'][j]['owner']['_id'] not in teachers['t_id']:
            data_teachers = {'t_id':str(course_count + 1).zfill(5)
                            , 't_name':result['data'][j]['owner']['name']}
        teachers = teachers.append(data_teachers, ignore_index = True)
        #funding
        try:
            data_funding = {'f_id':str(funding_count + 1).zfill(5)
                            , 'f_due_date':result['data'][j]['campaign']['expireDate']
                            , 'c_id':str(course_count + 1).zfill(5)}
            funding = funding.append(data_funding, ignore_index = True)
            funding_count += 1
        except KeyError:
            pass
        #discounts
        data_discounts = {'c_id':str(course_count + 1).zfill(5)
                        , 'd_id':str(discount_count + 1).zfill(5)
                        , 'd_content':'preOrderedPrice:' + str(result['data'][j]['basePricingInfo']['preOrderedPrice'])}
        discounts = discounts.append(data_discounts, ignore_index = True)
        discount_count += 1
        course_count += 1

In [77]:
for i in range(len(courses)):
    url = courses['c_url'][i]
    request = req.Request(url, headers = {
            "Content-Type":"application/json; charset=utf-8",
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
        })
    with req.urlopen(request) as response:
        result = response.read().decode("utf-8")
    result = json.loads(result)

    courses['c_total_time'][i] = result['estimatedCourseVideoLengthInMins']
    
    try:
        if result['group']['title'] == '語言':
            courses['cg_id'][i] = '01'
        elif result['group']['title'] == '藝術':
            courses['cg_id'][i] = '02'
        elif result['group']['title'] == '設計':
            courses['cg_id'][i] = '03'
        elif result['group']['title'] == '程式':
            courses['cg_id'][i] = '05'
        elif result['group']['title'] == '行銷':
            courses['cg_id'][i] = '06'
        elif result['group']['title'] == '投資理財':
            courses['cg_id'][i] = '07'
        elif result['group']['title'] == '職場技能':
            courses['cg_id'][i] = '08'
        elif result['group']['title'] == '音樂' or result['group']['title'] == '攝影' or result['group']['title'] == '人文' or result['group']['title'] == '手作' or result['group']['title'] == '生活品味':
            courses['cg_id'][i] = '09'
    except KeyError:
        pass
    
    if result['status'] == 'SUCCESS':
        courses['status_id'][i] = '01'
    elif result['status'] == 'PUBLISHED':
        courses['status_id'][i] = '02'
    elif result['status'] == 'INCUBATING':
        courses['status_id'][i] = '03'
        
    try:
        courses['c_start_time'][i] = result['estimatedCourseStartTime']
    except KeyError:
        pass

    try:
        teachers['t_introduction'][i] = result['owner']['metaDescription']
    except KeyError:
        pass

In [79]:
for i in range(len(courses)):
    url = courses['c_url'][i] + "/comments?limit=20&page=0"
    request = req.Request(url, headers = {
            "Content-Type":"application/json; charset=utf-8",
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
        })
    with req.urlopen(request) as response:
        result = response.read().decode("utf-8")
    result = json.loads(result)
    
    for j in range(len(result)):
        data_reviews = {'c_id':courses['c_id'][i]
                        , 'r_id':str(review_count + 1).zfill(5)
                        , 'r_content':result[j]['content']}
        reviews = reviews.append(data_reviews, ignore_index = True)
        review_count += 1

## Combination

In [80]:
courses_final = courses_final.append(courses)
reviews_final = reviews_final.append(reviews)
teachers_final = teachers_final.append(teachers)
funding_final = funding_final.append(funding)
discounts_final = discounts_final.append(discounts)

# TibaMe

In [81]:
url = "https://api.tibame.com/courselib/user/courseList"

requestData = {
    "orderBy": "popular",
    "sortType": "DESC",
    "isPublic": 1,
    "numPerPage": 400,
    "isCategory": False,
    "isSubCategory": False
}

headers = {
    "Content-Type":"application/json; charset=utf-8",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
}

request = req.Request(url, headers = headers, data = json.dumps(requestData).encode("utf-8"))

In [82]:
with req.urlopen(request) as response:
    result = response.read().decode("utf-8")
#print(result)

result = json.loads(result)


## Getting Data

In [83]:
courses = pd.DataFrame(columns = ['c_id', 'c_name', 'w_id', 'c_description', 'c_price', 'c_total_time',
                                  't_id', 'c_start_time', 'c_buyers', 'cg_id', 'c_url', 'c_img_url', 'status_id'])
teachers = pd.DataFrame(columns = ['t_id', 't_name', 't_introduction'])
discounts = pd.DataFrame(columns = ['d_id', 'c_id', 'd_content'])
#reviews = pd.DataFrame(columns = ['r_id', 'r_stars', 'r_content', 'c_id'])
#funding = pd.DataFrame(columns = ['f_id', 'f_due_date', 'f_process', 'c_id'])

In [84]:
for i in range(342):
    url2 = " https://api-c2c.tibame.com/v1/c2c/user-courses/course/" + str(result['data']['results'][i]['uid']) + "/detail"
    headers2 = {
        "Content-Type":"application/json; charset=utf-8",
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    }
    request2 = req.Request(url2, headers = headers2)
    
    with req.urlopen(request2) as response2:
        result2 = response2.read().decode("utf-8")

    result2 = json.loads(result2)
    # print(result2['data']['data']) 

    c_data = {
        'c_id': str(course_count + 1).zfill(5),
        'c_name': result['data']['results'][i]['courseName'],
        'w_id': '02',
        'c_description': result['data']['results'][i]['brief'],
        'c_price': result['data']['results'][i]['price'],
        'c_total_time': result2['data']['data']['videoTime'], #sec
        't_id': str(course_count + 1).zfill(5),
        # 'c_start_time'
        'c_buyers': result['data']['results'][i]['userCount'],
        # 'cg_id': result['data']['results'][i]['categoryUid'],
        'c_url': 'https://www.tibame.com/course/' + str(result['data']['results'][i]['uid']),
        'c_img_url': result['data']['results'][i]['coverPic']
        # 'status_id'
    }
    courses = courses.append(c_data, ignore_index = True)
    
    t_data = {
        't_id': str(course_count + 1).zfill(5),
        't_name': result['data']['results'][i]['teacher']['nickname'],
        't_introduction': result2['data']['data']['teacher']['introContent']
    }
    teachers = teachers.append(t_data, ignore_index = True)
    
    '''w_data = {
        # 'w_id': result['data']['results'][i]['uid'],
        # 'w_name'
        # 'w_url'
    }
    website = website.append(w_data, ignore_index = True)'''
    
    if result['data']['results'][i]['discountPrice'] != 0:
        d_data = {
            'd_id': str(discount_count + 1).zfill(5),
            'c_id': str(course_count + 1).zfill(5),
            'd_content': result['data']['results'][i]['discountPrice']
        }
        discounts = discounts.append(d_data, ignore_index = True)
        discount_count += 1
    
    r_data = {
        # 'r_id'
        # 'r_stars' 
        # 'r_content'
        # 'c_id'
    }
    reviews = reviews.append(r_data, ignore_index = True)
    
    f_data = {
        # 'f_id'
        # 'f_due_date'
        # 'f_process'
        #'c_id'
    }
    funding = funding.append(f_data, ignore_index = True)
    
    course_count += 1

## Combination

In [85]:
courses_final = courses_final.append(courses)
teachers_final = teachers_final.append(teachers)
discounts_final = discounts_final.append(discounts)

# Result

In [86]:
courses_final

,c_id,c_name,w_id,c_description,c_price,c_total_time,t_id,c_start_time,c_buyers,cg_id,c_url,c_img_url,status_id
0,00001,J Team 小學堂－英雄聯盟爬分 10 大重點,01,想提升《英雄聯盟》牌位嗎？由 PLANET9 與電競戰隊 J Team 合作，帶來職業等級的...,990,100,00001,2021-12-31T16:00:00.000Z,1,09,https://api.hahow.in/api/courses/6130753a26d20...,https://images.api.hahow.in/images/614c237ddeb...,03
1,00002,Knitting好好玩！自己織圍巾、披肩、羊毛帽,01,輕鬆有趣的編織課，打造獨一無二的風格穿搭。不論是基本針法還是觀念小撇步，都是編織前你想知道的...,2480,150,00002,2022-01-25T16:00:00.000Z,44,09,https://api.hahow.in/api/courses/6107cd6c3cc7a...,https://images.api.hahow.in/images/6149d8fc57d...,03
2,00003,放下酒譜！跟著 Mars 飛向浩瀚無垠調酒宇宙,01,To Infinity｜\n是用現代科學技術與無限的想像力來重新詮釋每一杯經典調酒，不再只是...,3980,360,00003,2022-01-25T16:00:00.000Z,17,09,https://api.hahow.in/api/courses/61666a458fc5c...,https://images.api.hahow.in/images/617824e6b2d...,03
3,00004,籌碼入門手冊：數據拆解主力思維,01,或許我們都認為小資族與大戶資源懸殊，怎麼可能追上他們呢？\n那你就錯了！其實你擁有的比想像多...,3600,260,00004,2022-01-24T16:00:00.000Z,18,07,https://api.hahow.in/api/courses/611dbe96be4b2...,https://images.api.hahow.in/images/614ad1c2deb...,03
4,00005,人物專訪怎麼寫？訪綱、採訪、寫作心法,01,對採訪寫作有興趣，卻不得其門而入？遇到難聊的受訪者緊張到流手汗？採訪完後不知文章怎麼開頭、擔...,2280,200,00005,2022-01-23T16:00:00.000Z,30,09,https://api.hahow.in/api/courses/5fe6fe2c5a5ba...,https://images.api.hahow.in/images/614a042f77d...,03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,01026,【聽課+實況】第二屆百業千師計劃—大遠端時代的溝通、協作與管理,02,講者國際工作經理豐富，且創辦 Meet.jobs 後有來自 18 國家的企業和求職用戶。將透...,0,4054,01026,NaN,382,NaN,https://www.tibame.com/course/1046,https://cdn-static.tibame.com/course/1046/cove...,NaN
338,01027,Windows Container入門與實作,02,想知道Windows Container容器技術能為 .net 開發團隊解決什麼問題嗎？ 容...,0,4634,01027,NaN,1186,NaN,https://www.tibame.com/course/79,https://cdn-static.tibame.com/course/79/coverP...,NaN
339,01028,人資長曾華英親身教你履歷面試技巧,02,讓你了解主管的心 ！ 寫出讓人難以放手的履歷 ！,0,6463,01028,NaN,2090,NaN,https://www.tibame.com/course/54,https://cdn-static.tibame.com/course/54/coverP...,NaN
340,01029,Chatbot的趨勢與實務應用,02,此門課程影片為TibaMe在2017年邀請3位實力專家就ChatBot產業最新趨勢分析進行分...,0,8712,01029,NaN,1805,NaN,https://www.tibame.com/course/52,https://cdn-static.tibame.com/course/52/coverP...,NaN


In [87]:
reviews_final

,c_id,r_id,r_statrs,r_content
0,00001,00001,NaN,竟然有英雄聯盟課程也太酷了吧~
1,00003,00002,NaN,想請問是上課又飲酒嗎？ 謝謝
2,00004,00003,NaN,我過去都是看基本面研究股票，感覺賺的好慢，最近想試看看學習籌碼面的研究，不知道我這樣從基本面...
3,00004,00004,NaN,如果對股票投資完全沒有概念的股市菜鳥適合上這堂課嗎？會不會聽不懂？
4,00004,00005,NaN,１．課程上會教當大戶在甩轎的時候（上下波動整理），大戶的籌碼變化嗎？\n（因為常常一個回檔就...
...,...,...,...,...
15798,00687,15799,NaN,請問購買後，課程是線上看，還是提供載點?\n課程結束可以重複看嗎?
15799,00687,15800,NaN,所以如果連photoshop跟 Illustrator都不會的人是否比較不建議上這門課呢~?
15800,00688,15801,NaN,需要電腦操作嗎？
15801,00688,15802,NaN,請問，鋼筆工具，拉最後一個點時，要同時按甚麼鍵(windows)?


In [88]:
teachers_final

,t_id,t_name,t_introduction
0,00001,PLANET9,PLANET9提供全球玩家全方位的遊戲體驗，從「組隊」、「找教練」到「賽事舉辦」等多元化服務...
1,00002,May｜就是愛編織,May，專職玩毛線十多年，有一間靠近海的手染編織工作室，有植物，好咖啡，和機靈的狗朋友Oreo
2,00003,Mars x 三十創意,【Mars】 為「2021年World Class世界頂尖調酒大賽」台灣冠軍。Mars自學習...
3,00004,豹投資,豹投資，專業投資人的邏輯整合大數據分析，成為多方投資策略的專業選股系統網站。\n不管在財務面...
4,00005,陳芷儀（耳草人內容工作室）,陳芷儀，耳草人內容工作室創辦人暨內容總監。天秤座 O 型（如果有人在意），政大傳播所畢。寫各...
...,...,...,...
337,01026,TibaMe,TibaMe是緯創旗下教育品牌一緯育所創的新一代學習平台，是全台最大的數位跨域人才培育加器。...
338,01027,TibaMe,TibaMe是緯創旗下教育品牌一緯育所創的新一代學習平台，是全台最大的數位跨域人才培育加器。...
339,01028,TibaMe,TibaMe是緯創旗下教育品牌一緯育所創的新一代學習平台，是全台最大的數位跨域人才培育加器。...
340,01029,TibaMe,TibaMe是緯創旗下教育品牌一緯育所創的新一代學習平台，是全台最大的數位跨域人才培育加器。...


In [89]:
funding_final

,f_id,f_due_date,f_process,c_id
0,00001,2021-11-01T16:00:00.000Z,NaN,00001
1,00002,2021-11-26T16:00:00.000Z,NaN,00002
2,00003,2021-11-26T16:00:00.000Z,NaN,00003
3,00004,2021-11-25T16:00:00.000Z,NaN,00004
4,00005,2021-11-18T16:00:00.000Z,NaN,00007
...,...,...,...,...
207,00208,2018-07-02T16:00:00.000Z,NaN,00473
208,00209,2018-06-11T16:00:00.000Z,NaN,00479
209,00210,2018-08-22T16:00:00.000Z,NaN,00484
210,00211,2018-06-04T16:00:00.000Z,NaN,00486


In [90]:
discounts_final

,c_id,d_id,d_content
0,00001,00001,preOrderedPrice:790
1,00002,00002,preOrderedPrice:1980
2,00003,00003,preOrderedPrice:2985
3,00004,00004,preOrderedPrice:2580
4,00005,00005,preOrderedPrice:1680
...,...,...,...
4,00699,00693,2730
5,00702,00694,900
6,00725,00695,698
7,00738,00696,290
